Introduction to machine learning
What is machine learninfg?
- Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying instead on patterns and inference from data.
- It involves training models on large datasets to make predictions or decisions based on new data.
- Subset of AI that enables systems to learn from data and also experience without explicit programming them.\

Benefits of Machine Learning

1. Improved Decision Making: Machine learning algorithms can analyze large volumes of data quickly and accurately, providing insights that help organizations make informed decisions.
2. Automation of Repetitive Tasks: ML can automate routine tasks, freeing up human resources for more complex and creative work.
3. Enhanced Customer Experiences: By analyzing customer data, ML can help businesses personalize their offerings and improve customer satisfaction.
4. Predictive Analytics: Machine learning can identify trends and patterns in data, enabling organizations to anticipate future outcomes and make proactive decisions.
5. Cost Savings: By optimizing processes and improving efficiency, machine learning can lead to significant cost reductions for businesses.
6. Continuous Improvement: ML models can learn and adapt over time, improving their accuracy and effectiveness as they are exposed to more data.

Types of Machine Learning
1. Supervised Learning : In supervised learning, the model is trained on a labeled dataset, which means that each training example is paired with an output label. The model learns to map inputs to the correct output and can make predictions on new, unseen data.

    Types
    - Linear regression
    - Logistuc regression
    - Decision trees
    - Support vector machines(SVM)
    - K-nearest neighbours
    - Random Forest
    - Deep learning (CNN) : image classification

2. Unsupervised Learning : Unsupervised learning involves training a model on data without labeled responses. The model tries to learn the underlying structure or distribution in the data to identify patterns or groupings.

    Types
    - K-means clustering
    - Hierarchical clustering
   
3. Reinforcement Learning : Reinforcement learning is a type of machine learning where an agent learns to make decisions by taking actions in an environment to maximize a reward signal. The agent learns from the consequences of its actions rather than from explicit examples.
4. Semi-supervised Learning : Semi-supervised learning is a hybrid approach that combines labeled and unlabeled data for training. It is useful when acquiring a fully labeled dataset is expensive or time-consuming.

    

In [8]:
%pip install numpy matplotlib tensorflow keras scipy

#importing required libraries with explanations

import os  # For interacting with the operating system (e.g., file paths)
import numpy as np # type: ignore # For numerical operations and array handling
import matplotlib.pyplot as plt # type: ignore # For data visualization and plotting
import tensorflow as tf  # type: ignore # Main TensorFlow library for machine learning and deep learning
from tensorflow import keras  # type: ignore # High-level API for building and training models

# Model building and layers
from tensorflow.keras.models import Sequential # type: ignore   # For creating sequential neural network models
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout  # Common neural network layers

# Data preprocessing and augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For image data augmentation and preprocessing

# Model training utilities
from tensorflow.keras.optimizers import Adam  # Adam optimizer for training neural networks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  # Callbacks for early stopping and saving best models

# Pre-trained models
from tensorflow.keras.applications import VGG16  # Pre-trained VGG16 model for transfer learning

%matplotlib inline

Note: you may need to restart the kernel to use updated packages.


2025-06-11 12:03:03.566487: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 12:03:03.567146: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 12:03:03.570208: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 12:03:03.577958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749632583.591176   94965 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749632583.59

In [9]:
#Define constants
IMAGE_SIZE = (256, 256)  # Size to which images will be resized
BATCH_SIZE = 32  # Number of samples per gradient update
EPOCHS = 20  # Number of epochs to train the model
NUM_CLASSES = 2  # Number of classes for crop disease detection (Healthy vs Diseased)
ANIMAL_CLASSES = 3  # Number of classes for animal filtering (Dog, Cat, Human)
# Paths
DATASET_DIR = "datasets"  # Directory containing the dataset
MODEL_PATH = "agricure_model.h5"  # Path to save the trained model

In [10]:
#Creating a CNN model for classification
def create_model(input_shape, num_classes):
    """Create a CNN model for classification"""
    model = Sequential([
        #32 is for number of filters, (3, 3) is the kernel size
        #Conv2D is a convolutional layer that applies a number of filters to the input image
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#32 is for number of filters, (3, 3) is the kernel size
        #MaxPooling2D reduces the spatial dimensions of the output volume like width and height by taking the maximum value in each patch of the feature map
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),# Flatten converts the 2D matrix into a 1D vector
        Dropout(0.5),# Dropout is a regularization technique to prevent overfitting by randomly setting a fraction of input units to 0 at each update during training time
        # Dropout layer helps to prevent overfitting by randomly setting a fraction of input units to 0 at each update during training time
        # Dense layers are fully connected layers in a neural network
        Dense(512, activation='relu'),
        # The final layer is a Dense layer with softmax activation for multi-class classification
        # The number of units in this layer should match the number of classes in your dataset
        # Softmax activation function is used for multi-class classification problems
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [11]:
def train_crop_model():
    """Train the main crop disease detection model"""
    # Create an ImageDataGenerator with augmentation and validation split
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # 20% for validation
    )
    
    # Training generator (uses subset='training')
    train_generator = train_datagen.flow_from_directory(
        os.path.join(DATASET_DIR, "crops"),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',  # Important!
        shuffle=True
    )

    # Validation generator (uses subset='validation')
    validation_generator = train_datagen.flow_from_directory(
        os.path.join(DATASET_DIR, "crops"),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',  # Important!
        shuffle=False
    )

    # Verify we found images
    print(f"Found {train_generator.samples} training images")
    print(f"Found {validation_generator.samples} validation images")
    if train_generator.samples == 0 or validation_generator.samples == 0:
        raise ValueError("No images found. Check your dataset structure and paths.")

    # Create model
    model = create_model(IMAGE_SIZE + (3,), len(train_generator.class_indices))
    
    # Callbacks
    callbacks = [
        ModelCheckpoint(CROP_MODEL_PATH, save_best_only=True),
        EarlyStopping(patience=5, restore_best_weights=True)
    ]

    # Train
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        callbacks=callbacks
    )
    
    plot_training_history(history)
    return model

In [12]:
def train_animal_filter():
    """Train a secondary model to filter out animals/humans"""
    animal_path = os.path.join(DATASET_DIR, "animals")
    
    # Debugging checks
    if not os.path.exists(animal_path):
        raise FileNotFoundError(f"Animal directory not found at: {animal_path}")
    
    print("\nDataset contents:")
    for class_name in os.listdir(animal_path):
        class_path = os.path.join(animal_path, class_name)
        if os.path.isdir(class_path):
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            print(f"{class_name}: {len(images)} images")
            if len(images) == 0:
                print(f"WARNING: No images found in {class_path}")

    animal_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = animal_datagen.flow_from_directory(
        animal_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    validation_generator = animal_datagen.flow_from_directory(
        animal_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    print(f"\nTraining images: {train_generator.samples}")
    print(f"Validation images: {validation_generator.samples}")
    
    if train_generator.samples == 0:
        raise ValueError("No training images found. Check that:")
        print("- Images exist in the subdirectories")
        print("- Images are in .jpg, .png, or .jpeg format")
        print(f"- Directory structure: {animal_path}/classname/[images]")

    model = create_model(IMAGE_SIZE + (3,), len(train_generator.class_indices))
    
    model.fit(
        train_generator,
        steps_per_epoch=max(1, train_generator.samples // BATCH_SIZE),
        epochs=10,
        validation_data=validation_generator,
        validation_steps=max(1, validation_generator.samples // BATCH_SIZE)
    )

    return model

In [13]:
def plot_training_history(history):
    """Plot training and validation accuracy/loss"""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.savefig('training_history.png')
    plt.close()

In [14]:
def train_animal_filter():
    """Train a secondary model to filter out animals/humans"""
    animal_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2  # 20% for validation
    )

    # Training generator
    train_generator = animal_datagen.flow_from_directory(
        os.path.join(DATASET_DIR, "animals"),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    # Validation generator
    validation_generator = animal_datagen.flow_from_directory(
        os.path.join(DATASET_DIR, "animals"),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    # Verify we found images
    print(f"Found {train_generator.samples} training images (animals)")
    print(f"Found {validation_generator.samples} validation images (animals)")
    if train_generator.samples == 0 or validation_generator.samples == 0:
        raise ValueError("No animal images found. Check your dataset structure and paths.")

    # Create model (number of classes is automatically determined from subdirectories)
    model = create_model(IMAGE_SIZE + (3,), len(train_generator.class_indices))

    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE
    )

    return model

In [15]:
if __name__ == "__main__":
    # Define model paths (add these to your constants)
    CROP_MODEL_PATH = "models/crop_model.h5"
    ANIMAL_MODEL_PATH = "models/animal_model.h5"
    
    # Create models directory if it doesn't exist
    os.makedirs("models", exist_ok=True)

    # Train or load models
    if not os.path.exists(CROP_MODEL_PATH) or not os.path.exists(ANIMAL_MODEL_PATH):
        print("Training models...")
        
        print("\nTraining crop disease model...")
        crop_model = train_crop_model()
        crop_model.save(CROP_MODEL_PATH)
        print(f"Crop model saved to {CROP_MODEL_PATH}")
        
        print("\nTraining animal filter model...")
        animal_model = train_animal_filter()
        animal_model.save(ANIMAL_MODEL_PATH)
        print(f"Animal model saved to {ANIMAL_MODEL_PATH}")
    else:
        print("Loading existing models...")
        try:
            crop_model = tf.keras.models.load_model(CROP_MODEL_PATH)
            animal_model = tf.keras.models.load_model(ANIMAL_MODEL_PATH)
            print("Models loaded successfully")
        except Exception as e:
            print(f"Error loading models: {e}")
            print("Retraining models...")
            crop_model = train_crop_model()
            animal_model = train_animal_filter()

    # Test prediction
    test_image = "test_image.jpg"  # Replace with your test image
    if os.path.exists(test_image):
        try:
            prediction = predict_image(crop_model, animal_model, test_image)
            print("\nPrediction Results:")
            print(f"Type: {prediction['type']}")
            print(f"Class: {prediction['class']}")
            print(f"Confidence: {prediction['confidence']:.2%}")
            
            # For debugging class mappings:
            print("\nClass Indices:")
            print("Crop classes:", crop_model.class_indices if hasattr(crop_model, 'class_indices') else "Not available")
            print("Animal classes:", animal_model.class_indices if hasattr(animal_model, 'class_indices') else "Not available")
        except Exception as e:
            print(f"Prediction failed: {e}")
    else:
        print(f"Test image {test_image} not found")
        print("Current directory contents:", os.listdir('.'))

Training models...

Training crop disease model...
Found 60 images belonging to 2 classes.
Found 13 images belonging to 2 classes.
Found 60 training images
Found 13 validation images


/home/ltgwgeorge/.local/share/virtualenvs/ltgwgeorge-VLsDEmAT/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
E0000 00:00:1749632602.738303   94965 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1749632602.738663   94965 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/ho

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6071 - loss: 0.6856

/home/ltgwgeorge/.local/share/virtualenvs/ltgwgeorge-VLsDEmAT/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6071 - loss: 0.6856 - val_accuracy: 0.4615 - val_loss: 0.8081
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 740ms/step - accuracy: 0.3750 - loss: 0.8445

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8750 - loss: 0.3857 - val_accuracy: 1.0000 - val_loss: 0.1405
Epoch 19/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8571 - loss: 0.3176

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8571 - loss: 0.3176 - val_accuracy: 1.0000 - val_loss: 0.1373
Epoch 20/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 750ms/step - accuracy: 0.8750 - loss: 0.3719

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8750 - loss: 0.3719 - val_accuracy: 1.0000 - val_loss: 0.1287


Crop model saved to models/crop_model.h5

Training animal filter model...
Found 7 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 7 training images (animals)
Found 0 validation images (animals)


ValueError: No animal images found. Check your dataset structure and paths.